In [1]:
!pip install datasets
!pip install peft
!pip install evaluate
!pip install -U "huggingface_hub[cli]"
! pip install -U accelerate
! pip install -U transformers
! pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 32.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 27.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-n

In [2]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    BitsAndBytesConfig,
    Trainer,
    HfArgumentParser)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import bitsandbytes as bnb

In [3]:

!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 

Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

# dataset

In [4]:
# sst2
# The Stanford Sentiment Treebank consists of sentences from movie reviews and human annotations of their sentiment. The task is to predict the sentiment of a given sentence. It uses the two-way (positive/negative) class split, with only sentence-level labels.
# dataset = load_dataset('csv', data_dir='/sem.csv', split='train')
dataset = load_dataset("sudan94/SemEvalEmoji2018Binary")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'label', 'tweet'],
        num_rows: 3555
    })
    validation: Dataset({
        features: ['index', 'label', 'tweet'],
        num_rows: 444
    })
    test: Dataset({
        features: ['index', 'label', 'tweet'],
        num_rows: 445
    })
})

In [5]:
# display % of training data with label=1
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

0.5037974683544304

# model

In many cases, the architecture you want to use can be guessed from the name or the path of the pretrained model you are supplying to the from_pretrained method.

AutoClasses are here to do this job for you so that you automatically retrieve the relevant model given the name/path to the pretrained weights/config/vocabulary:

Instantiating one of AutoModel, AutoConfig and AutoTokenizer will directly create a class of the relevant architecture (ex: model = AutoModel.from_pretrained('bert-base-cased') will create a instance of BertModel).

In [6]:

model_checkpoint = 'google-bert/bert-large-cased'

compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype='int8',
)

# define label maps
id2label = {0: "non_ironic", 1: "irony"}
label2id = {"non_ironic":0, "irony":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id,
    config=bnb_config,
)

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# from peft import prepare_model_for_kbit_training

# #gradient checkpointing to save memory
# model.gradient_checkpointing_enable()

# model=prepare_model_for_kbit_training(
#     model, use_gradient_checkpointing=False
# )
# model.get_memory_footprint()
# print(model)

In [8]:
# display architecture
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1

# preprocess data

Tokenization is a critical first step in preparing data for Large Language Models (LLMs) because these models don't understand raw text; they process numerical data. The tokenizer's role is to convert text into numbers that the model can understand.

In [9]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding_side="left")

# add pad token if none exists
# Pad Token (pad_token): In NLP, padding is used to ensure that all sequences (like sentences or paragraphs) are of the same length when feeding them into a model.
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [10]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["tweet"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text, #actual text
        return_tensors="np", # return type numpy array
        truncation=True, #Indicates that truncation should be applied based on the specified parameters
        max_length=512 #Specifies the maximum length of the tokenized sequence
    )

    return tokenized_inputs

In [11]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/3555 [00:00<?, ? examples/s]

Map:   0%|          | 0/444 [00:00<?, ? examples/s]

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'label', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3555
    })
    validation: Dataset({
        features: ['index', 'label', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 444
    })
    test: Dataset({
        features: ['index', 'label', 'tweet', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 445
    })
})

In [12]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# evaluation

In [13]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

In [14]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    accuray = accuracy.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    # print(accuracy)

    return {"accuray": accuray['accuracy'], "f1": f1['f1']}

    # return {"accuracy":accuracy.compute(predictions=predictions, references=labels), "f1":f1_metric.compute(predictions=predictions, references=labels,average="macro")}

# Apply untrained model to text

In [15]:
# define list of examples
text_list = dataset["test"]["tweet"]
ground_truth = dataset["test"]["label"]
# Initialize a list to store the table data
table_data = []
i = 0
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)
    # print(predictions.tolist())

     # Get the label string
    predicted_label = id2label[predictions.tolist()]
    # Append the text and predicted label to the table data
    table_data.append([text, predicted_label,predictions.tolist(), ground_truth[i]])
    i+=1

    # print(text + " - " + id2label[predictions.tolist()])


In [16]:
# print(table_data)
import pandas as pd
df = pd.DataFrame(table_data,columns =['tweet','predicted_class','predicted_label','actual_label'])
df.head(5)

,tweet,predicted_class,predicted_label,actual_label
0,@restlessduncan @SR_Duncan I accept him the wa...,non_ironic,0,0
1,Happy stfx day! Lost my #Xring in '12. It was ...,non_ironic,0,1
2,K. Michelle said Lil Kim is Plastic but 65% of...,non_ironic,0,1
3,doesn't convey what I want though. #twitterpr...,non_ironic,0,0
4,It's hard to take people seriously who can't s...,non_ironic,0,0


In [17]:
# from tabulate import tabulate
# print("Untrained model predictions:")
# print("----------------------------")
# print(tabulate(table_data, headers=["tweet", "predicted_class", "predicted_label" ,"actual_label"], tablefmt="grid"))

# Train model

In [18]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=64,
                        lora_alpha=4,
                        lora_dropout=0.1, # dropot rate for avoiding overfitting
                        target_modules = ['query'])

In [19]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=64, target_modules={'query'}, lora_alpha=4, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [20]:
# model.float()

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 3,147,778 || all params: 336,729,092 || trainable%: 0.9348


In [21]:
# hyperparameters
lr = 1e-3
# lr = 1e-4 #default learning rate
batch_size = 16
num_epochs = 10


In [22]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    logging_steps = 223,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model = 'eval_f1'
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuray,F1
1,0.743800,0.782890,0.531532,0.460312
2,0.668700,0.644157,0.635135,0.635543
3,0.601500,0.622411,0.668919,0.667045
4,0.567600,0.584302,0.682432,0.682767
5,0.523200,0.570875,0.693694,0.693694
6,0.502500,0.592238,0.716216,0.716493
7,0.465700,0.621551,0.680180,0.677149
8,0.450300,0.583306,0.713964,0.712740
9,0.431400,0.594570,0.707207,0.707492
10,0.411000,0.614287,0.704955,0.705217


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=2230, training_loss=0.5365678727359515, metrics={'train_runtime': 408.7975, 'train_samples_per_second': 86.962, 'train_steps_per_second': 5.455, 'total_flos': 3683279606313600.0, 'train_loss': 0.5365678727359515, 'epoch': 10.0})

# Generate prediction

In [24]:
best_checkpoint = trainer.state.best_model_checkpoint
best_checkpoint


'google-bert/bert-large-cased-lora-text-classification/checkpoint-1338'

In [25]:
trainer.state

TrainerState(epoch=10.0, global_step=2230, max_steps=2230, logging_steps=223, eval_steps=500, save_steps=500, train_batch_size=16, num_train_epochs=10, num_input_tokens_seen=0, total_flos=3683279606313600.0, log_history=[{'loss': 0.7438, 'grad_norm': 5.82206916809082, 'learning_rate': 0.0009000000000000001, 'epoch': 1.0, 'step': 223}, {'eval_loss': 0.7828904390335083, 'eval_accuray': 0.5315315315315315, 'eval_f1': 0.4603116630143657, 'eval_runtime': 2.2956, 'eval_samples_per_second': 193.411, 'eval_steps_per_second': 12.197, 'epoch': 1.0, 'step': 223}, {'loss': 0.6687, 'grad_norm': 9.281538009643555, 'learning_rate': 0.0008, 'epoch': 2.0, 'step': 446}, {'eval_loss': 0.6441574096679688, 'eval_accuray': 0.6351351351351351, 'eval_f1': 0.635543319492631, 'eval_runtime': 2.361, 'eval_samples_per_second': 188.055, 'eval_steps_per_second': 11.859, 'epoch': 2.0, 'step': 446}, {'loss': 0.6015, 'grad_norm': 6.816276550292969, 'learning_rate': 0.0007, 'epoch': 3.0, 'step': 669}, {'eval_loss': 0.6

In [26]:
f = open("trainer-output.txt", "w")
f.write(str(trainer.state))
f.close()
trainer.state

TrainerState(epoch=10.0, global_step=2230, max_steps=2230, logging_steps=223, eval_steps=500, save_steps=500, train_batch_size=16, num_train_epochs=10, num_input_tokens_seen=0, total_flos=3683279606313600.0, log_history=[{'loss': 0.7438, 'grad_norm': 5.82206916809082, 'learning_rate': 0.0009000000000000001, 'epoch': 1.0, 'step': 223}, {'eval_loss': 0.7828904390335083, 'eval_accuray': 0.5315315315315315, 'eval_f1': 0.4603116630143657, 'eval_runtime': 2.2956, 'eval_samples_per_second': 193.411, 'eval_steps_per_second': 12.197, 'epoch': 1.0, 'step': 223}, {'loss': 0.6687, 'grad_norm': 9.281538009643555, 'learning_rate': 0.0008, 'epoch': 2.0, 'step': 446}, {'eval_loss': 0.6441574096679688, 'eval_accuray': 0.6351351351351351, 'eval_f1': 0.635543319492631, 'eval_runtime': 2.361, 'eval_samples_per_second': 188.055, 'eval_steps_per_second': 11.859, 'epoch': 2.0, 'step': 446}, {'loss': 0.6015, 'grad_norm': 6.816276550292969, 'learning_rate': 0.0007, 'epoch': 3.0, 'step': 669}, {'eval_loss': 0.6

In [29]:
model.to('cpu')
# Initialize a list to store the table data
table_data = []
i=0
print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu")

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    # Get the label string
    predicted_label = id2label[predictions.tolist()[0]]
    # Append the text and predicted label to the table data
    table_data.append((text, predicted_label,predictions.tolist()[0], ground_truth[i]))
    i+=1

    # print(text + " - " + id2label[predictions.tolist()[0]])
tuned_df = pd.DataFrame(table_data,columns =['tweet','predicted_class','predicted_label','actual_label'])
tuned_df.head(5)


Trained model predictions:
--------------------------


,tweet,predicted_class,predicted_label,actual_label
0,@restlessduncan @SR_Duncan I accept him the wa...,non_ironic,0,0
1,Happy stfx day! Lost my #Xring in '12. It was ...,irony,1,1
2,K. Michelle said Lil Kim is Plastic but 65% of...,irony,1,1
3,doesn't convey what I want though. #twitterpr...,irony,1,0
4,It's hard to take people seriously who can't s...,non_ironic,0,0


In [30]:
nottuned = "nottuned_result_bert_set1_qlora.csv"
finetuned = "finetuned_result_bert_set1_qlora.csv"

df.to_csv(nottuned,  encoding='utf-8')
tuned_df.to_csv(finetuned, encoding='utf-8')


In [ ]:
# model.save_pretrained('fine_tuned_model')